# Stoneburner, Kurt
- ## DSC 540 - Week 11/12

In [1]:
# //****************************************************************************************
# //*** Set Working Directory to thinkstats folder.
# //*** This pseudo-relative path call should work on all Stoneburner localized projects. 
# //****************************************************************************************

import os
import sys
# //*** Imports and Load Data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sqlite3

### Task 1: (Connect to the supplied petsDB, and (OPTIONAL) write a function to check if the connection is done)

In [31]:
#//*** Returns True if database connection is open. Otherise returns False. I liberally borrowed (ie Stole)
#//*** from the Inter-webs
def chk_conn(conn):
     try:
        conn.cursor()
        return True
     except Exception as ex:
        return False
    
db_filename = 'z_sup_wk10_11_petsdbz'

#//*** Verify database instance exists
if os.path.exists(db_filename) == False:
    #//*** Throw a warning if databse does not exist
    print(f"Trouble Connecting to database: {db_filename}")
else:
    #//*** Start a database connection
    conn = sqlite3.connect(db_filename)
    
    #//*** Verify database connection
    if chk_conn(conn):
        print("Database Connected....")    

Database Connected....


### Task 2: (What are the different age groups in the persons database)

In [197]:
### Write your code below this comment
cursor = conn.cursor()

#//*** Get a list of all the tables in the database.
#//*** Print a list of the columns in each table for reference
#//*** We could look these up in the book. But how does that help us in the wild?

for results in conn.execute("SELECT name FROM sqlite_master WHERE type='table';"):
    table = results[0]
    print("========================")
    print(f"Table: {table}")
    print("========================")
    print("Columns:")


    #//*** Read from the table, this modfies the cursor object, which will provide column names
    for col in cursor.execute(f'SELECT * FROM {table}').description:
        print(f"\t{col[0]}")

    print("")


Table: persons
Columns:
	id
	first_name
	last_name
	age
	city
	zip_code

Table: pets
Columns:
	owner_id
	pet_name
	pet_type
	treatment_done



In [59]:
#//*** Use the GROUP BY query to the unique values in the age column
#//*** Equivalent to pandas df['column'].unique()
ages = cursor.execute('SELECT age FROM persons GROUP BY age')
print("Unique Ages")
for age in ages:
    print(age[0])

Unique Ages
5
6
7
8
9
11
12
13
14
16
17
18
19
22
23
24
25
27
30
31
32
33
34
35
36
37
39
40
42
44
48
49
50
51
52
53
54
58
59
60
61
62
63
65
66
67
68
69
70
71
72
73
74


### Task 3: Which age group has maximum number of people?

In [84]:
#//*** Get the age column, group by age (unique age values), order the values by the total number of unique values
#//*** In ascending order. The last Item will have the most
rows = cursor.execute('SELECT age FROM persons GROUP BY age ORDER BY count(age)')

#//*** Get the last item from the cursor group
#//*** Convert the response to a list and grab the last value.
#//*** There is the possibility of multiple max values, and we'd need a list to accomodate that scenario
#//*** But it does not apply here
max_age = list(rows)[-1][0]

#//*** Get the count of the group by ages and sort them by the count. The last value will be the corresponding counts of the
#//*** Max_age value. Why? Because we can.
rows = cursor.execute('SELECT count(age) FROM persons GROUP BY age ORDER BY count(age)')
#//*** Get the last value returned.
max_count = list(rows)[-1][0]

print(f"The Largest age group is: {max_age} with {max_count} people")

columns = "first_name, last_name, age, city, zip_code"
rows = cursor.execute(f'SELECT {columns} FROM persons WHERE age={max_age}')

for row in rows:
    print(f"Name: {row[0]} {row[1]}\n{row[3]}, {row[4]}\nAge: {row[2]}\n")

The Largest age group is: 73 with 5 people
Name: Jordi None
east port, 123456
Age: 73

Name: Cary None
new port, 76856785
Age: 73

Name: Pearline None
new port, 9756543
Age: 73

Name: Maritza None
east port, 9756543
Age: 73

Name: Jaquelin None
west port, 9756543
Age: 73



### Task 4: How many people do not have a full name (Last name is blank/null)

In [124]:
rows = cursor.execute(f'SELECT count(*) FROM persons WHERE last_name IS NULL')
print(f"There are {rows.fetchall()[0][0]} customers with no last name.\n")

columns = "first_name, last_name, age, city, zip_code"
rows = cursor.execute(f'SELECT {columns} FROM persons WHERE last_name IS NULL')
counter = 0
for row in rows:
    counter += 1
    spacer = "   "
    
    if counter > 9:
        spacer += " "
    print(f"{counter}. {row[0]} {row[1]}\n{spacer}{row[3]}, {row[4]}\n{spacer}Age: {row[2]}\n")

There are 60 customers with no last name.

1. Erica None
   south port, 2345678
   Age: 22

2. Jordi None
   east port, 123456
   Age: 73

3. Chasity None
   new port, 76856785
   Age: 70

4. Gregg None
   new port, 76856785
   Age: 31

5. Cary None
   new port, 76856785
   Age: 73

6. Francisca None
   west port, 123456
   Age: 14

7. Raleigh None
   new port, 2345678
   Age: 68

8. Maria None
   west port, 123456
   Age: 42

9. Mariane None
   south port, 9756543
   Age: 62

10. Mona None
    south port, 76856785
    Age: 44

11. Kayla None
    south port, 2345678
    Age: 36

12. Karlie None
    west port, 123456
    Age: 35

13. Morris None
    west port, 76856785
    Age: 71

14. Sandy None
    east port, 2345678
    Age: 23

15. Hector None
    east port, 9756543
    Age: 63

16. Hiram None
    west port, 2345678
    Age: 52

17. Tressa None
    new port, 123456
    Age: 59

18. Berry None
    south port, 2345678
    Age: 22

19. Pearline None
    new port, 9756543
    Age: 73

2

### Task 5: How many people has more than one pet? (*)

In [230]:
#//*** Nested Count.
#//*** The first select count(*) counts the total results from the subsequent group by query
#//*** Group By owner_id, count the instances of owner_if that have a count the owner_ids greater than one.
#//*** This query assumes that each pet is unique and we are not counting multiple owner visits with the same animal
#//*** We could further sort for duplicate animal names, but we are not going to. Well, at least not in pure SQL
for results in cursor.execute(f'SELECT count(*) FROM (SELECT count(*) FROM pets GROUP BY owner_id HAVING count(owner_id)>1)'):
    print(f"There are {results[0]}")

counter = 0

#//*** Print The details of each owner along with their pets' names. Requires multiple separate queries because I'm not
#/*** sure how to join multiple pet names for the same owner. 

for results in cursor.execute(f'SELECT * FROM pets GROUP BY owner_id HAVING count(owner_id)>1'):
    owner_id = results[0]
    
    #//*** Get the animal names based on the owner_id
    animals = conn.execute(f'SELECT pet_name FROM pets WHERE owner_id = {owner_id}').fetchall()
    
    #//*** Get the owner info from the persons table based on owner_id
    owner_info = conn.execute(f'SELECT * FROM persons WHERE id = {owner_id}').fetchall()[0]
    
    counter += 1
    spacer = "   "
    
    if counter > 9:
        spacer += " "

    fur_babies = ""
    
    #//*** Parse the animal names into a string
    for fur_baby in animals:
        fur_babies += f"{fur_baby[0]}, "
    
    #//*** Display the info
    print(f"{counter}. {owner_info[1]} {owner_info[2]}\n{spacer}{owner_info[3]}, {owner_info[4]}\n{spacer}Age: {owner_info[2]}\n{spacer}Pets: {fur_babies[:-2]}\n")


There are 43
1. Jordi None
   73, east port
   Age: None
   Pets: mani, gimir, tamari

2. Tony Lindgren
   7, west port
   Age: Lindgren
   Pets: fenga, dara, dara

3. Cary None
   73, new port
   Age: None
   Pets: milu, deru

4. Gabe Thompson
   54, new port
   Age: Thompson
   Pets: olga, unsa, helu

5. Katelyn Torphy
   49, east port
   Age: Torphy
   Pets: gimir, palu

6. Raleigh None
   68, new port
   Age: None
   Pets: snoopy, nani, sapi

7. Mona None
   44, south port
   Age: None
   Pets: gimir, dara

8. Kayla None
   36, south port
   Age: None
   Pets: unsa, chegal

9. Morris None
   71, west port
   Age: None
   Pets: dara, fenga

10. Hector None
    63, east port
    Age: None
    Pets: deru, milu

11. Tressa None
    59, new port
    Age: None
    Pets: olga, fenga

12. Berry None
    22, south port
    Age: None
    Pets: tamari, chegal, mani

13. Dorian None
    40, east port
    Age: None
    Pets: palu, fenga

14. Mylene None
    5, east port
    Age: None
    Pets: 

### Task 6: How many pets have received treaments?


In [186]:
#//*******************************************
#//*** Running out of commenting gas.....
#//*******************************************
#//*** Get a count of pets where treatment_done = 1 which indicates treatment
for results in conn.execute(f'SELECT count(*) FROM pets where treatment_done=1'):
    print(f"There are {results[0]} pets that have received treatment")


There are 36 pets that have received treatment


### Task 7: How many pets have received treatment that we know the type of?

In [192]:
#//*** Get a count of pets where treatment_done = 1 which indicates treatment and pet_type is null/empty
for results in conn.execute(f'SELECT count(*) FROM pets where treatment_done=1 AND pet_type IS NULL'):
    print(f"There are {results[0]} pets that have received treatment and have an unknown type.")


There are 20 pets that have received treatment and have an unknown type.


### Task 8: How many pets are there from the city called "east port"

In [210]:
#//*** Get a count of the result of a table join which connects to tables
#//*** FROM persons JOIN pets <-- combines results from Persons and pets tables
#//*** ON persons.id = pets.owner_id <-- Links Persons Table Id column with Pets Table owner_id column. These need to be 
#//***       shared values linking the two tables
for results in conn.execute(f'SELECT count(*) FROM (SELECT * FROM persons JOIN pets ON persons.id = pets.owner_id WHERE persons.city="east port")'):
    print(f"There are {results[0]} pets located in the city of east port.")

for results in conn.execute(f'SELECT count(*) FROM (SELECT count(*) FROM persons JOIN pets ON persons.id = pets.owner_id GROUP BY persons.id HAVING persons.city="east port")'):    #print(f"There are {results[0]} pets that have received treatment and have an unknown type.")
    print(f"The are {results[0]} pet owners located in the city of east port")


There are 49 pets located in the city of east port.
The are 21 pet owners located in the city of east port


### Task 9: How many pets are there from the city called "east port" and who received a treatment?

In [224]:
#//*** Fine I'll acquiesce to building queries as a separate string. The SQL is getting a bit complex,
#//*** or as I like to call it "Overly ridiculous".
#//*** Same syntax as in Task 8 but with the added criteria of treatment_done = 1
query = """
    SELECT COUNT(*) FROM
        (
        SELECT * FROM persons
        JOIN pets ON persons.id = pets.owner_id 
        WHERE persons.city="east port"
        AND treatment_done=1
        )
"""

#print(query)
#print(conn.execute(query).fetchall())
for results in conn.execute(query):
    print(f"There are {results[0]} pets that have owners from east port and have had treatment")
#    print(f"There are {results[0]} pets located in the city of east port.")


There are 11 pets that have owners from east port and have had treatment


In [ ]:
#//*** Close and Exit the database connection
#//*** Because "we" gracefully close our database connections. We clean up our messes and practice good work habits
con.close()
con.__exit__